<a href="https://colab.research.google.com/github/be-mich/LIGN-167-Project/blob/main/Back_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip uninstall googletrans

#necessary to run for correct google translate version.

This notebook conducts backtranslation on some of the Seinfeld data. Backtranslation is when a sentence is translated to one language and then back to it's original language. The goal of backtranslation, and of this notebook is to conduct data augmentation. Once a sentence is backtranslated, the original context is kept but the order and exact phrasing is different. Data augmentation can help our model learn more. Backtranslation is only done on a sample of the data because of computational resource limitations (google api limitations). 

In [ ]:
#correct version of google translate
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 1.3 MB 52.7 MB/s 
     |████████████████████████████████| 65 kB 3.9 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=aab7d39f26d295884f07d8410a5e78aef2bc23a6b9579e34f3b8625fd9edec13
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
from googletrans import Translator

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
#initalization 
GEN_BACK_TR = True

GEN_UPSAMPLE = False

GEN_EN_ONLY = False

In [ ]:
!pip install dask[bag]

     |████████████████████████████████| 134 kB 10.5 MB/s 


In [ ]:
#for parrallel processing 

from dask import bag, diagnostics 

In [ ]:
def back_translate(sequence, PROB = 1):
  languages = ['en', 'fr','th','tr','ur','ru',
               'bg','de','ar','zh-cn','hi',
               'sw','vi','es','el']
  translator = Translator()

  org_lang = translator.detect(sequence).lang

  random_lang = np.random.choice([lang for lang in languages if lang is not org_lang])

  if org_lang in languages:
    translated = translator.translate(sequence, dest = random_lang).text

    translated_back = translator.translate(translated, dest = org_lang).text

    if np.random.uniform(0, 1) <= PROB:
      output_sequence = translated_back
    else:
      output_sequence = sequence

  else:
      output_sequence = sequence
  return output_sequence

for i in range(5):
  output = back_translate('I genuinely have no idea what the output of this sequence of words will be')
  print(output)


I really do not know what the outcome of this sequence of words will be
I really don't know What will the outcome of this word sequence be?
I don't really know what the output of this sequence of words is
I have no idea what the outcome of this sequence of words will be.
I really have no idea the outcome of this string of words


I will run backtranslation on a data sample with a context window of size 10. So i prepare the back translation function to read and apply the back translate function to each of these columns.

In [ ]:
#use parallel processing for above function 

def back_translate_parallel(dataset):
  response_bag = bag.from_sequence(dataset['response'].tolist()).map(back_translate)
  context_bag = bag.from_sequence(dataset['context'].tolist()).map(back_translate)
  context_zero_bag = bag.from_sequence(dataset['context/0'].tolist()).map(back_translate)
  context_one_bag = bag.from_sequence(dataset['context/1'].tolist()).map(back_translate)
  context_second_bag = bag.from_sequence(dataset['context/2'].tolist()).map(back_translate)
  context_third_bag = bag.from_sequence(dataset['context/3'].tolist()).map(back_translate)
  context_fourth_bag = bag.from_sequence(dataset['context/4'].tolist()).map(back_translate)
  context_fifth_bag =  bag.from_sequence(dataset['context/5'].tolist()).map(back_translate)
  context_sixth_bag = bag.from_sequence(dataset['context/6'].tolist()).map(back_translate)
  context_seventh_bag = bag.from_sequence(dataset['context/7'].tolist()).map(back_translate)
  context_eigth_bag = bag.from_sequence(dataset['context/8'].tolist()).map(back_translate)


  with diagnostics.ProgressBar():
    response = response_bag.compute()
    context = context_bag.compute()
    context_zero = context_zero_bag.compute()
    context_one = context_one_bag.compute()
    context_second = context_second_bag.compute()
    context_third = context_third_bag.compute()
    context_fourth = context_fourth_bag.compute()
    context_fifth = context_fifth_bag.compute()
    context_sixth = context_sixth_bag.compute()
    context_seventh = context_seventh_bag.compute()
    context_eigth = context_eigth_bag.compute()

  #put it all together 
  #dataset[['response']] = list(zip(response))
  dataset[['response','context','context_zero','context_one','context_second',
           'context_third','context_fourth', 'context_fifth',
           'context_sixth','context_seventh','context_eigth']] = list(zip(response, context, context_zero,
                                                                          context_one, context_second, 
                                                                          context_third, context_fourth, 
                                                                          context_fifth,
                                                                         context_sixth, context_seventh,
                                                                          context_eigth))
  return dataset

In [ ]:
#read in seinfeld 
#setup 
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
! gdown https://drive.google.com/uc?id=1wSKl_642G6lsk4AmrYXpa_8XWb4hcY_B

Downloading...
From: https://drive.google.com/uc?id=1wSKl_642G6lsk4AmrYXpa_8XWb4hcY_B
To: /content/scripts.csv
100% 4.52M/4.52M [00:00<00:00, 140MB/s]


In [ ]:
seinfeld_scripts = pd.read_csv("./scripts.csv")

In [ ]:
#drop rows with missing values

seinfeld_scripts = seinfeld_scripts.dropna().reset_index(drop=True)
print(len(seinfeld_scripts))

54606


In [ ]:
#can get rid of some columns- we only need character and dialogue
seinfeld_scripts = seinfeld_scripts.drop(columns=['Unnamed: 0','EpisodeNo','SEID','Season'])

In [ ]:
NAME = 'GEORGE'

In [ ]:
contexted = []
#context window of size 4
#or window of 10
#n = 4
n = 10

for i in seinfeld_scripts[seinfeld_scripts.Character == NAME].index:
    if i < n:
        continue
    row = []
    prev = i - 1 - n #subtract 1 so row contains current and 4/ or 10 previous responses
    for j in range(i, prev, -1):
        row.append(seinfeld_scripts.Dialogue[j])
    contexted.append(row)
columns = ['response','context']
columns = columns + ['context/' + str(i) for i in range(n-1)]

formatted_df = pd.DataFrame.from_records(contexted, columns=columns)
    

In [ ]:
formatted_df.sample(6)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
195,Why did you give him my ticket for?,"Cmon, cmon, go ahead, move up.",No! My ticket?! You gave my ticket to Horneck?,"Yeah, Im sorry. I had to give it to Horneck.",(not believing him) You what?!,I gave your ticket to Horneck.,What?,"The thing is, yesterday, I kind of.. uh..",What about it?,"Listen, George, little problem with the game.",I got scared there for a second. The Knicks wi...
6008,"Ma, don't cry!","Oh, I was so worried. (grabs a couple of tissu...",Yeah.,The proctologist? Are you okay?,"You know, the corkscrew pasta. It was a Fusill...",Fusilli?,I fell on some Fusilli.,Where have you been?!! You were supposed to fi...,"It was a million to one shot, Doc. Million to ...",So...you're the Assman.,Yes. I'm still waiting for the motor vehicle b...
5360,Jon Voight! Are you kiddin' me? Did you talk t...,No. (To George) But I did run into somebody yo...,The way I feel when I go places with you? (Kra...,"Jerry, for all I know this guy went out of his...",You don't want to go with me?,Maybe I'll just meet you there.,"Yeah, I gotta see a dentist, this is killin' m...",You don't want to suffer with this all weekend.,"Yeah, you're right.","Well, there's going to be a lot of dentists th...","No, I don't know if I'm invited."
2620,"What, are you crazy? It's a traumatic thing to...",Look just because Kramer vomited on her doesn'...,You know she was behind the idea. She was goin...,I think SHE saw it coming.,I never should have brought her up there. Shou...,Noooo... (Kramer throws up on Susan),"Um, Uh, ugh, ...",and SUSAN The third?,The third?,September third.,What's the expiration date on that?
9352,I knew I had hit my high note so I thanked the...,Good luck with that.,The surgeon left a sponge inside me.,"Okay, what do you got?",Sure.,"Sorry buddy, this is the ""Hamlet"" of diseases....",Gonorrhea? You wanna trade?,Bacterial Meningitis. Jackpot!,In your packet you will find the disease you h...,(getting up and leaving) Alright! That's it fo...,"Alright, well why don't we smooth the head dow..."
1832,"No, I won't.","Oh, yes you will.",Why even try anymore? There's no sense to it. ...,OH! I got to call her back.,Elle McPherson going to be there?,"Um, Id like to but a bunch of us from the isla...",Were taking Elaine to dinner. Shes got to star...,"Hey, where ya going?",All right were coming down.,Its Elaine.,Yeah.


In [ ]:
#sample some of the data. The data is too large to do backtranslation on all of it.

seinfeld_sample = formatted_df.sample(frac = 0.1, random_state=42)

In [ ]:
seinfeld_sample.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
483,Tough.,"So, is there any way out of this Elaine thing?",My censor quit two years ago. He checked into ...,My censoring system broke down. You know that ...,Two minutes.,They saw me walk out.,"Oh, whos gonna know?",I was just in there. Its embarrassing.,Come in for two minutes and sit with me.,Of course. We all do.,But I need the home bed advantage.
7506,I like it!,"Yes, it is.",That's pretty paranoid.,"I heard that when Jerry Lewis left a meeting, ...",Jerry Lewis?,"You know, if you really want to test him out, ...",Do you really think I'm wrong about this Wyck ...,"So, Elaine was telling me about this piece of ...",Look at him! Just look at him!,"No, no, that's okay, it's uh...",Oh! I have pictures!
8795,Like retouched.,You now what you do? You sneak that photo out ...,It is a horrible company. There's no managemen...,"Right, they couldn't get the green stuff off.","Grinders, sanders, wet stones. They are the on...",Yeah.,Kruger? That's not Kruger Industrial Smoothing...,Until that stupid photo jogs his memory.,And that guy is your new boss?,"So now, the father is screaming at me, he's de...",Even more reasonable.
1688,"Eh, gee, I hope you have the key for these thi...",Why couldn't I take a cab. For 6 dollars my wh...,It's a deal!,Tell you what. If they win the penant I'll sit...,I LUV their chances.,But you gotta like their chances.,They got Franco! ...What they need is a front ...,Franco's no good? ...They got no team leaders.,Speed? They got Coleman. ...They need a bullpen.,Defence? Please. ...They need speed.,"No hitting? They got hitting! Bonilla, Murry. ..."
6217,How did you know that?,Let me take a guess. She cried and you caved.,"Yeah, yeah, sure, Christmas. Snow. Santa. All ...",Are you sure?,"No no no, no mistake, no mistake. No, no , lis...",My parents told me you were too neurotic and t...,Sstill love. Still love.,You don't love me.,"Still marry , still marry.","If you don't want to marry me, George, just sa...","No, no no it's not about postponing. I just th..."


In [ ]:
print(len(seinfeld_sample))

970


In [ ]:
#take part of this sample

first_sample = seinfeld_sample.sample(frac=0.5, random_state=42)
print(len(first_sample))

485


In [ ]:
if GEN_BACK_TR:
  first_back_tr = first_sample.pipe(back_translate_parallel)

  first_back_tr.to_csv('first_back_tr.csv')

[########################################] | 100% Completed |  4min 27.7s
[########################################] | 100% Completed |  4min 24.8s
[########################################] | 100% Completed |  4min 26.1s
[########################################] | 100% Completed |  4min 28.9s
[########################################] | 100% Completed |  4min 29.8s
[########################################] | 100% Completed |  4min 26.3s
[########################################] | 100% Completed |  4min 29.3s
[########################################] | 100% Completed |  4min 27.0s
[########################################] | 100% Completed |  4min 28.6s
[########################################] | 100% Completed |  4min 29.3s
[########################################] | 100% Completed |  4min 25.5s


In [ ]:
first_back_tr

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8,context_zero,context_one,context_second,context_third,context_fourth,context_fifth,context_sixth,context_seventh,context_eigth
2304,"Alright, stop it Kramer, you're freakin' me ou...","Oh, yeah. I can barely see you, George.",Me?,"Because you see, George, having the keys to Je...",I suppose I could.,"Well, you can get 'em back.","Should I give you my keys, is that the transac...","Say yes! Yes, George. Yes!","Gee, Kramer, I uh...I don't know what to say.","Yeah. Yeah, I think it would be for the best.",You want 'em back?,Me?,"Because you see, George, having the keys to Je...",I suppose I could.,"Well, you can get 'em back.","Should I give you my keys, is that the transac...","Say yes! Yes, George. Yes!","Gee, Kramer, I uh...I don't know what to say.","Yeah. Yeah, I think it would be for the best.",You want 'em back?
4450,You nailed it,he asked me,horning on my rock climbing trip. It's just su...,what?,hey nice move today,hey,he's the first cool guy I've ever been friends...,"cool guy? what are you, in 8th grade?",yeah that's right. I like it. He's such a cool...,yeah yeah what is it with you and Tony? what a...,alright alright,horning on my rock climbing trip. It's just su...,what?,hey nice move today,hey,he's the first cool guy I've ever been friends...,"cool guy? what are you, in 8th grade?",yeah that's right. I like it. He's such a cool...,yeah yeah what is it with you and Tony? what a...,alright alright
4560,cinnamon. Why didn't Cha get chocolate?,Cinnamon babka.,"I spilled some Chardonnay. So, what did you get?",What for?,I had to give it to the liquor store guy.,"Hey, what happened to your coat? And what is t...",Somebody put a cane on my foot. Just like the ...,"What, what happened to you?","S-s-somebody double parked, we couldn't help i...","How about a nice box of ""scram"".","Oh, no thanks.","I spilled some Chardonnay. So, what did you get?",What for?,I had to give it to the liquor store guy.,"Hey, what happened to your coat? And what is t...",Somebody put a cane on my foot. Just like the ...,"What, what happened to you?","S-s-somebody double parked, we couldn't help i...","How about a nice box of ""scram"".","Oh, no thanks."
8918,it's a pleasure to meet you.,"(Introducing the two) George, this is Nina.",Yeah!,It's been years!,Hi.,Nina?,Jerry?,Five' eight. Five' seven.,Really?,"Yeah, and a whole new me. I'm up two inches in...","Almost. (Notices his shoes) Hey, new Timberlands?",Yeah!,It's been years!,Hi.,Nina?,Jerry?,Five' eight. Five' seven.,Really?,"Yeah, and a whole new me. I'm up two inches in...","Almost. (Notices his shoes) Hey, new Timberlands?"
5581,Hey.,(entering) Hey.,"Many times. You love velvet, you want to live ...",I've said that before?,"I know, you would drape yourself in velvet.","I gotta find a way to work this out, I love th...",We'll take a check please.,Ho ho. This bizarre ?harrod? experiment must end!,Not you.,I look like me and I'm working from the outsid...,He looks like you and he's working from the in...,"Many times. You love velvet, you want to live ...",I've said that before?,"I know, you would drape yourself in velvet.","I gotta find a way to work this out, I love th...",We'll take a check please.,Ho ho. This bizarre ?harrod? experiment must end!,Not you.,I look like me and I'm working from the outsid...,He looks like you and he's working from the in...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9280,When are they gonna learn that any news about ...,"Well, this is one for the books, huh, Jerry?.....",You mean it was open? We desecrated a pet ceme...,"Oh. Well, would you look at that. (puts a gun-...","So, what's in the cooler? (flips open the lid)","(still in tuxedo, tie undone) Ah, I'll just ta...","(to Jerry) So, what are you gonna do? Are you ...","Give me forty, you'll never see me again.",All right. I'm gonna try givin' them fifty-fiv..

Check why there are duplicated columns, and see if they are in fact duplicated.

In [ ]:
first_back_tr['context/0'].equals(first_back_tr['context_zero'])

True

Lets save this first half of back translated data.

In [ ]:
first_back_tr.to_csv('/content/drive/My Drive/Colab Notebooks/first_back_tr_sample.csv')

In [ ]:
first_back_tr = pd.read_csv("first_back_tr_sample.csv")

In [ ]:
#lets get other half 

sample_all = seinfeld_sample.merge(first_back_tr, #on=['response'],
                                   left_index = True, 
                                   right_index = True,
                                   how = 'left', indicator = True)

sample_all

,response_x,context_x,context/0_x,context/1_x,context/2_x,context/3_x,context/4_x,context/5_x,context/6_x,context/7_x,...,context_zero,context_one,context_second,context_third,context_fourth,context_fifth,context_sixth,context_seventh,context_eigth,_merge
483,Tough.,"So, is there any way out of this Elaine thing?",My censor quit two years ago. He checked into ...,My censoring system broke down. You know that ...,Two minutes.,They saw me walk out.,"Oh, whos gonna know?",I was just in there. Its embarrassing.,Come in for two minutes and sit with me.,Of course. We all do.,...,kissed me. We both had tears streaming down ou...,(quietly) Um-hm.,"Ohh yes, yes. Yes Jerry they were all over me....",Well the-the crowd must have gone wild!,A hole in one eh.,(Silently squinches his mouth and nods at Kramer),What is that a Titleist?,"Well then, from out of nowhere, a huge tidal w...","Well then, what did you do next?",both
7506,I like it!,"Yes, it is.",That's pretty paranoid.,"I heard that when Jerry Lewis left a meeting, ...",Jerry Lewis?,"You know, if you really want to test him out, ...",Do you really think I'm wrong about this Wyck ...,"So, Elaine was telling me about this piece of ...",Look at him! Just look at him!,"No, no, that's okay, it's uh...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8795,Like retouched.,You now what you do? You sneak that photo out ...,It is a horrible company. There's no managemen...,"Right, they couldn't get the green stuff off.","Grinders, sanders, wet stones. They are the on...",Yeah.,Kruger? That's not Kruger Industrial Smoothing...,Until that stupid photo jogs his memory.,And that guy is your new boss?,"So now, the father is screaming at me, he's de...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1688,"Eh, gee, I hope you have the key for these thi...",Why couldn't I take a cab. For 6 dollars my wh...,It's a deal!,Tell you what. If they win the penant I'll sit...,I LUV their chances.,But you gotta like their chances.,They got Franco! ...What they need is a front ...,Franco's no good? ...They got no team leaders.,Speed? They got Coleman. ...They need a bullpen.,Defence? Please. ...They need speed.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6217,How did you know that?,Let me take a guess. She cried and you caved.,"Yeah, yeah, sure, Christmas. Snow. Santa. All ...",Are you sure?,"No no no, no mistake, no mistake. No, no , lis...",My parents told me you were too neurotic and t...,Sstill love. Still love.,You don't love me.,"Still marry , still marry.","If you don't want to marry me, George, just sa...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,I mean it's hard enough to meet a woman you di...,"Yeah, maybe you won't.","No, I won't.","Oh, yes you will.",Why even try anymore? There's no sense to it. ...,OH! I got to call her back.,Elle McPherson going to be there?,"Um, Id like to but a bunch of us from the isla...",Were taking Elaine to dinner. Shes got to star...,"Hey, where ya going?",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5846,(enthusiastic) Yeah. You get a couch. I get ri...,(not sure) I guess.,"No, the cushion's turned over.",But it's got a pee-stain on it.,"Yeah, sure. (big smile) Then my father will ha...",The one with the Poppie stain?!,(crafty) Not necessarily. Why don't you take b...,Now we have to buy a new couch?!,We have to replace the couch.,"Anyway, Jerry... Jerry?",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3688,Why?,I don't know about defraying.,"Hey, guess what? The Drake broke up.",Hey.,"Don't worry, they'll make friends fast with th...",They don't know anybody in Chicago.,I hate the Drake! Maybe the whole thing was a ...,"Boy, I am really starting to dislike the Drake!",I know it is!,"Well, she can't keep it, it's not fair, that's...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
620,Revenge.,What is the point of all this?,But she didn't know he does

Lets get all observations from the 10% sample that are not contained in the 5% subset.

In [ ]:
sample_two = sample_all[sample_all['_merge'] == 'left_only']

In [ ]:
print(len(sample_two))

485


In [ ]:
sample_two

,response_x,context_x,context/0_x,context/1_x,context/2_x,context/3_x,context/4_x,context/5_x,context/6_x,context/7_x,...,context_zero,context_one,context_second,context_third,context_fourth,context_fifth,context_sixth,context_seventh,context_eigth,_merge
7506,I like it!,"Yes, it is.",That's pretty paranoid.,"I heard that when Jerry Lewis left a meeting, ...",Jerry Lewis?,"You know, if you really want to test him out, ...",Do you really think I'm wrong about this Wyck ...,"So, Elaine was telling me about this piece of ...",Look at him! Just look at him!,"No, no, that's okay, it's uh...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6217,How did you know that?,Let me take a guess. She cried and you caved.,"Yeah, yeah, sure, Christmas. Snow. Santa. All ...",Are you sure?,"No no no, no mistake, no mistake. No, no , lis...",My parents told me you were too neurotic and t...,Sstill love. Still love.,You don't love me.,"Still marry , still marry.","If you don't want to marry me, George, just sa...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7109,(gleeful) Hehee. W..we could be like the Gatsb...,Whatever.,"Well, she's smart. You take my word for it.","Well, I dunno. I didn't really talk to her.","She's got everything, right? (counts on his fi...",She's got 'it' written all over her.,(claps hands triumphantly) We might have an 'it'!,She could be an 'it'.,You think she could be an 'it'? Could she be a...,I'm nuts about her.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2698,All right. I don't.,"Well, either you see him or you don't.",I'm not* sure.,Do you see him?,(Strangling Kramer) What's the matter with you...,That'll be seventy-five dollars.,(to Kramer) You shut up! (to judge) I demand a...,I'm distraught!?! Wooh-wooh-hoo!,"Your Honor, Mister Kramer's obviously very dis...",That's enough already.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9606,Can I go like this?,"""Jerry""!","""Jerry""!","He wants to talk about ""Jerry""!","He wants to talk about ""Jerry""?","""Jerry""!","Right now? ""Jerry""?","Today, like right now.",When?,Yeah!,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,It's the pesto of cities. So..?,Everybody's moving to Seattle.,Seattle.,"He's from a.. Yakima, right?","(Slightly embarrassed) Yes, a guy.",A guy?,"Elaine is having a ""houseguest."" She's picking...",Who?,I don't understand why he couldn't take a cab.,That's a tough minute. It's like waiting in th...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1071,That's pretty long.,About three years.,So how long did you live there?,"On my block, a lot of ah, people walk their do...","Well, thank you very much. I'm telling you, on...",It's just we want you to go.,"I know, it's not that.","Forget it. Go ahead, you'll have a good time.",What about this--,No.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5846,(enthusiastic) Yeah. You get a couch. I get ri...,(not sure) I guess.,"No, the cushion's turned over.",But it's got a pee-stain on it.,"Yeah, sure. (big smile) Then my father will ha...",The one with the Poppie stain?!,(crafty) Not necessarily. Why don't you take b...,Now we have to buy a new couch?!,We have to replace the couch.,"Anyway, Jerry... Jerry?",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3688,Why?,I don't know about defraying.,"Hey, guess what? The Drake broke up.",Hey.,"Don't worry, they'll make friends fast with th...",They don't know anybody in Chicago.,I hate the Drake! Maybe the whole thing was a ...,"Boy, I am really starting to dislike the Drake!",I know it is!,"Well, she can't keep it, it's not fair, that's...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [ ]:
print(sample_two.columns.values.tolist())

['response_x', 'context_x', 'context/0_x', 'context/1_x', 'context/2_x', 'context/3_x', 'context/4_x', 'context/5_x', 'context/6_x', 'context/7_x', 'context/8_x', 'response_y', 'context_y', 'context/0_y', 'context/1_y', 'context/2_y', 'context/3_y', 'context/4_y', 'context/5_y', 'context/6_y', 'context/7_y', 'context/8_y', 'context_zero', 'context_one', 'context_second', 'context_third', 'context_fourth', 'context_fifth', 'context_sixth', 'context_seventh', 'context_eigth', '_merge']


In [ ]:
#rename column
sample_two = sample_two.rename(columns = {'response_x':'response','context_x':'context','context/0_x':'context/0',
                                          'context/1_x':'context/1','context/2_x':'context/2',
                                          'context/3_x':'context/3','context/4_x':'context/4',
                                          'context/5_x':'context/5','context/6_x':'context/6',
                                          'context/7_x':'context/7','context/8_x':'context/8'})

sample_two

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,...,context_zero,context_one,context_second,context_third,context_fourth,context_fifth,context_sixth,context_seventh,context_eigth,_merge
7506,I like it!,"Yes, it is.",That's pretty paranoid.,"I heard that when Jerry Lewis left a meeting, ...",Jerry Lewis?,"You know, if you really want to test him out, ...",Do you really think I'm wrong about this Wyck ...,"So, Elaine was telling me about this piece of ...",Look at him! Just look at him!,"No, no, that's okay, it's uh...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6217,How did you know that?,Let me take a guess. She cried and you caved.,"Yeah, yeah, sure, Christmas. Snow. Santa. All ...",Are you sure?,"No no no, no mistake, no mistake. No, no , lis...",My parents told me you were too neurotic and t...,Sstill love. Still love.,You don't love me.,"Still marry , still marry.","If you don't want to marry me, George, just sa...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7109,(gleeful) Hehee. W..we could be like the Gatsb...,Whatever.,"Well, she's smart. You take my word for it.","Well, I dunno. I didn't really talk to her.","She's got everything, right? (counts on his fi...",She's got 'it' written all over her.,(claps hands triumphantly) We might have an 'it'!,She could be an 'it'.,You think she could be an 'it'? Could she be a...,I'm nuts about her.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2698,All right. I don't.,"Well, either you see him or you don't.",I'm not* sure.,Do you see him?,(Strangling Kramer) What's the matter with you...,That'll be seventy-five dollars.,(to Kramer) You shut up! (to judge) I demand a...,I'm distraught!?! Wooh-wooh-hoo!,"Your Honor, Mister Kramer's obviously very dis...",That's enough already.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9606,Can I go like this?,"""Jerry""!","""Jerry""!","He wants to talk about ""Jerry""!","He wants to talk about ""Jerry""?","""Jerry""!","Right now? ""Jerry""?","Today, like right now.",When?,Yeah!,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,It's the pesto of cities. So..?,Everybody's moving to Seattle.,Seattle.,"He's from a.. Yakima, right?","(Slightly embarrassed) Yes, a guy.",A guy?,"Elaine is having a ""houseguest."" She's picking...",Who?,I don't understand why he couldn't take a cab.,That's a tough minute. It's like waiting in th...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1071,That's pretty long.,About three years.,So how long did you live there?,"On my block, a lot of ah, people walk their do...","Well, thank you very much. I'm telling you, on...",It's just we want you to go.,"I know, it's not that.","Forget it. Go ahead, you'll have a good time.",What about this--,No.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5846,(enthusiastic) Yeah. You get a couch. I get ri...,(not sure) I guess.,"No, the cushion's turned over.",But it's got a pee-stain on it.,"Yeah, sure. (big smile) Then my father will ha...",The one with the Poppie stain?!,(crafty) Not necessarily. Why don't you take b...,Now we have to buy a new couch?!,We have to replace the couch.,"Anyway, Jerry... Jerry?",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3688,Why?,I don't know about defraying.,"Hey, guess what? The Drake broke up.",Hey.,"Don't worry, they'll make friends fast with th...",They don't know anybody in Chicago.,I hate the Drake! Maybe the whole thing was a ...,"Boy, I am really starting to dislike the Drake!",I know it is!,"Well, she can't keep it, it's not fair, that's...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [ ]:
#keep certain columns only 

sample_two = sample_two[['response','context','context/0','context/1',
                         'context/2','context/3','context/4','context/5',
                         'context/6','context/7','context/8']]

sample_two

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
7506,I like it!,"Yes, it is.",That's pretty paranoid.,"I heard that when Jerry Lewis left a meeting, ...",Jerry Lewis?,"You know, if you really want to test him out, ...",Do you really think I'm wrong about this Wyck ...,"So, Elaine was telling me about this piece of ...",Look at him! Just look at him!,"No, no, that's okay, it's uh...",Oh! I have pictures!
6217,How did you know that?,Let me take a guess. She cried and you caved.,"Yeah, yeah, sure, Christmas. Snow. Santa. All ...",Are you sure?,"No no no, no mistake, no mistake. No, no , lis...",My parents told me you were too neurotic and t...,Sstill love. Still love.,You don't love me.,"Still marry , still marry.","If you don't want to marry me, George, just sa...","No, no no it's not about postponing. I just th..."
7109,(gleeful) Hehee. W..we could be like the Gatsb...,Whatever.,"Well, she's smart. You take my word for it.","Well, I dunno. I didn't really talk to her.","She's got everything, right? (counts on his fi...",She's got 'it' written all over her.,(claps hands triumphantly) We might have an 'it'!,She could be an 'it'.,You think she could be an 'it'? Could she be a...,I'm nuts about her.,I told you.
2698,All right. I don't.,"Well, either you see him or you don't.",I'm not* sure.,Do you see him?,(Strangling Kramer) What's the matter with you...,That'll be seventy-five dollars.,(to Kramer) You shut up! (to judge) I demand a...,I'm distraught!?! Wooh-wooh-hoo!,"Your Honor, Mister Kramer's obviously very dis...",That's enough already.,"A banker! A banker! Your Honor, Your Honor, Yo..."
9606,Can I go like this?,"""Jerry""!","""Jerry""!","He wants to talk about ""Jerry""!","He wants to talk about ""Jerry""?","""Jerry""!","Right now? ""Jerry""?","Today, like right now.",When?,Yeah!,"""Jerry"", oh my God. He wants to talk about ""Je..."
...,...,...,...,...,...,...,...,...,...,...,...
856,It's the pesto of cities. So..?,Everybody's moving to Seattle.,Seattle.,"He's from a.. Yakima, right?","(Slightly embarrassed) Yes, a guy.",A guy?,"Elaine is having a ""houseguest."" She's picking...",Who?,I don't understand why he couldn't take a cab.,That's a tough minute. It's like waiting in th...,"(To Elaine) Oh, one more thing about the car. ..."
1071,That's pretty long.,About three years.,So how long did you live there?,"On my block, a lot of ah, people walk their do...","Well, thank you very much. I'm telling you, on...",It's just we want you to go.,"I know, it's not that.","Forget it. Go ahead, you'll have a good time.",What about this--,No.,Oh! I tell you what. How about if I come back ...
5846,(enthusiastic) Yeah. You get a couch. I get ri...,(not sure) I guess.,"No, the cushion's turned over.",But it's got a pee-stain on it.,"Yeah, sure. (big smile) Then my father will ha...",The one with the Poppie stain?!,(crafty) Not necessarily. Why don't you take b...,Now we have to buy a new couch?!,We have to replace the couch.,"Anyway, Jerry... Jerry?",And they think they're better than us?
3688,Why?,I don't know about defraying.,"Hey, guess what? The Drake broke up.",Hey.,"Don't worry, they'll make friends fast with th...",They don't know anybody in Chicago.,I hate the Drake! Maybe the whole thing was a ...,"Boy, I am really starting to dislike the Drake!",I know it is!,"Well, she can't keep it, it's not fair, that's...",He gave her all the gifts; he felt guilty.


In [ ]:
#back translation on this half of data 

if GEN_BACK_TR:
  back_tr_second = sample_two.pipe(back_translate_parallel)

  back_tr_second.to_csv('back_tr_second.csv')

[########################################] | 100% Completed |  4min 26.4s
[########################################] | 100% Completed |  4min 26.8s
[########################################] | 100% Completed |  4min 27.3s
[########################################] | 100% Completed |  4min 23.9s
[########################################] | 100% Completed |  4min 28.1s
[########################################] | 100% Completed |  4min 30.5s
[########################################] | 100% Completed |  4min 25.5s
[########################################] | 100% Completed |  4min 26.9s
[########################################] | 100% Completed |  4min 27.0s
[########################################] | 100% Completed |  4min 26.4s
[########################################] | 100% Completed |  4min 23.2s


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [ ]:
back_tr_second

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8,context_zero,context_one,context_second,context_third,context_fourth,context_fifth,context_sixth,context_seventh,context_eigth
7506,I like it!,"Yes, it is.",That's pretty paranoid.,"I heard that when Jerry Lewis left a meeting, ...",Jerry Lewis?,"You know, if you really want to test him out, ...",Do you really think I'm wrong about this Wyck ...,"So, Elaine was telling me about this piece of ...",Look at him! Just look at him!,"No, no, that's okay, it's uh...",Oh! I have pictures!,That's pretty paranoid.,"I heard that when Jerry Lewis left a meeting, ...",Jerry Lewis?,"You know, if you really want to test him out, ...",Do you really think I'm wrong about this Wyck ...,"So, Elaine was telling me about this piece of ...",Look at him! Just look at him!,"No, no, that's okay, it's uh...",Oh! I have pictures!
6217,How did you know that?,Let me take a guess. She cried and you caved.,"Yeah, yeah, sure, Christmas. Snow. Santa. All ...",Are you sure?,"No no no, no mistake, no mistake. No, no , lis...",My parents told me you were too neurotic and t...,Sstill love. Still love.,You don't love me.,"Still marry , still marry.","If you don't want to marry me, George, just sa...","No, no no it's not about postponing. I just th...","Yeah, yeah, sure, Christmas. Snow. Santa. All ...",Are you sure?,"No no no, no mistake, no mistake. No, no , lis...",My parents told me you were too neurotic and t...,Sstill love. Still love.,You don't love me.,"Still marry , still marry.","If you don't want to marry me, George, just sa...","No, no no it's not about postponing. I just th..."
7109,(gleeful) Hehee. W..we could be like the Gatsb...,Whatever.,"Well, she's smart. You take my word for it.","Well, I dunno. I didn't really talk to her.","She's got everything, right? (counts on his fi...",She's got 'it' written all over her.,(claps hands triumphantly) We might have an 'it'!,She could be an 'it'.,You think she could be an 'it'? Could she be a...,I'm nuts about her.,I told you.,"Well, she's smart. You take my word for it.","Well, I dunno. I didn't really talk to her.","She's got everything, right? (counts on his fi...",She's got 'it' written all over her.,(claps hands triumphantly) We might have an 'it'!,She could be an 'it'.,You think she could be an 'it'? Could she be a...,I'm nuts about her.,I told you.
2698,All right. I don't.,"Well, either you see him or you don't.",I'm not* sure.,Do you see him?,(Strangling Kramer) What's the matter with you...,That'll be seventy-five dollars.,(to Kramer) You shut up! (to judge) I demand a...,I'm distraught!?! Wooh-wooh-hoo!,"Your Honor, Mister Kramer's obviously very dis...",That's enough already.,"A banker! A banker! Your Honor, Your Honor, Yo...",I'm not* sure.,Do you see him?,(Strangling Kramer) What's the matter with you...,That'll be seventy-five dollars.,(to Kramer) You shut up! (to judge) I demand a...,I'm distraught!?! Wooh-wooh-hoo!,"Your Honor, Mister Kramer's obviously very dis...",That's enough already.,"A banker! A banker! Your Honor, Your Honor, Yo..."
9606,Can I go like this?,"""Jerry""!","""Jerry""!","He wants to talk about ""Jerry""!","He wants to talk about ""Jerry""?","""Jerry""!","Right now? ""Jerry""?","Today, like right now.",When?,Yeah!,"""Jerry"", oh my God. He wants to talk about ""Je...","""Jerry""!","He wants to talk about ""Jerry""!","He wants to talk about ""Jerry""?","""Jerry""!","Right now? ""Jerry""?","Today, like right now.",When?,Yeah!,"""Jerry"", oh my God. He wants to talk about ""Je..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,It's the pesto of cities. So..?,Everybody's moving to Seattle.,Seattle.,"He's from a.. Yakima, right?","(Slightly embarrassed) Yes, a guy.",A guy?,"Elaine is having a ""houseguest."" She's picking...",Who?,I don't understand why he couldn't take a cab.,That's a tough minute. It's like waiting in th...,"(To El

Save this back translated data.

In [ ]:
back_tr_second.to_csv('/content/drive/My Drive/Colab Notebooks/back_tr_second.csv')

In [ ]:
back_tr_second = pd.read_csv("back_tr_second.csv")

In [ ]:
#concat the two back translated data 

back_translated_ten_per = pd.concat([first_back_tr,back_tr_second], ignore_index = True)

#back_translated_response = back_translated_response[['response']]

back_translated_ten_per

,Unnamed: 0,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,...,context/8,context_zero,context_one,context_second,context_third,context_fourth,context_fifth,context_sixth,context_seventh,context_eigth
0,2304,"Alright, stop it Kramer, you're freakin' me ou...","Oh, yeah. I can barely see you, George.",Me?,"Because you see, George, having the keys to Je...",I suppose I could.,"Well, you can get 'em back.","Should I give you my keys, is that the transac...","Say yes! Yes, George. Yes!","Gee, Kramer, I uh...I don't know what to say.",...,You want 'em back?,Me?,"Because you see, George, having the keys to Je...",I suppose I could.,"Well, you can get 'em back.","Should I give you my keys, is that the transac...","Say yes! Yes, George. Yes!","Gee, Kramer, I uh...I don't know what to say.","Yeah. Yeah, I think it would be for the best.",You want 'em back?
1,4450,You nailed it,he asked me,horning on my rock climbing trip. It's just su...,what?,hey nice move today,hey,he's the first cool guy I've ever been friends...,"cool guy? what are you, in 8th grade?",yeah that's right. I like it. He's such a cool...,...,alright alright,horning on my rock climbing trip. It's just su...,what?,hey nice move today,hey,he's the first cool guy I've ever been friends...,"cool guy? what are you, in 8th grade?",yeah that's right. I like it. He's such a cool...,yeah yeah what is it with you and Tony? what a...,alright alright
2,4560,cinnamon. Why didn't Cha get chocolate?,Cinnamon babka.,"I spilled some Chardonnay. So, what did you get?",What for?,I had to give it to the liquor store guy.,"Hey, what happened to your coat? And what is t...",Somebody put a cane on my foot. Just like the ...,"What, what happened to you?","S-s-somebody double parked, we couldn't help i...",...,"Oh, no thanks.","I spilled some Chardonnay. So, what did you get?",What for?,I had to give it to the liquor store guy.,"Hey, what happened to your coat? And what is t...",Somebody put a cane on my foot. Just like the ...,"What, what happened to you?","S-s-somebody double parked, we couldn't help i...","How about a nice box of ""scram"".","Oh, no thanks."
3,8918,it's a pleasure to meet you.,"(Introducing the two) George, this is Nina.",Yeah!,It's been years!,Hi.,Nina?,Jerry?,Five' eight. Five' seven.,Really?,...,"Almost. (Notices his shoes) Hey, new Timberlands?",Yeah!,It's been years!,Hi.,Nina?,Jerry?,Five' eight. Five' seven.,Really?,"Yeah, and a whole new me. I'm up two inches in...","Almost. (Notices his shoes) Hey, new Timberlands?"
4,5581,Hey.,(entering) Hey.,"Many times. You love velvet, you want to live ...",I've said that before?,"I know, you would drape yourself in velvet.","I gotta find a way to work this out, I love th...",We'll take a check please.,Ho ho. This bizarre ?harrod? experiment must end!,Not you.,...,He looks like you and he's working from the in...,"Many times. You love velvet, you want to live ...",I've said that before?,"I know, you would drape yourself in velvet.","I gotta find a way to work this out, I love th...",We'll take a check please.,Ho ho. This bizarre ?harrod? experiment must end!,Not you.,I look like me and I'm working from the outsid...,He looks like you and he's working from the in...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,856,It's the pesto of cities. So..?,Everybody's moving to Seattle.,Seattle.,"He's from a.. Yakima, right?","(Slightly embarrassed) Yes, a guy.",A guy?,"Elaine is having a ""houseguest."" She's picking...",Who?,I don't understand why he couldn't take a cab.,...,"(To Elaine) Oh, one more thing about the car. ...",Seattle.,"He's from a.. Yakima, right?","(Slightly embarrassed) Yes, a guy.",A guy?,"Elaine is having a ""houseguest."" She's picking...",Who?,I don't understand why he couldn't take a cab.,That's a tough minute. It's like waiting in th...,"(To Elaine) Oh, one more thing about the car. ..."
966,1071,That's pretty long.,About three ye

In [ ]:
#dataframe twice as wide...lets keep only necessary columns 

back_translated_tenpercent_sample = back_translated_ten_per.drop(columns=['context_zero','context_one','context_second','context_third',
                                                                'context_fourth','context_fifth','context_sixth',
                                                                'context_seventh','context_eigth'])

In [ ]:
back_translated_tenpercent_sample

,Unnamed: 0,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6,context/7,context/8
0,2304,"Alright, stop it Kramer, you're freakin' me ou...","Oh, yeah. I can barely see you, George.",Me?,"Because you see, George, having the keys to Je...",I suppose I could.,"Well, you can get 'em back.","Should I give you my keys, is that the transac...","Say yes! Yes, George. Yes!","Gee, Kramer, I uh...I don't know what to say.","Yeah. Yeah, I think it would be for the best.",You want 'em back?
1,4450,You nailed it,he asked me,horning on my rock climbing trip. It's just su...,what?,hey nice move today,hey,he's the first cool guy I've ever been friends...,"cool guy? what are you, in 8th grade?",yeah that's right. I like it. He's such a cool...,yeah yeah what is it with you and Tony? what a...,alright alright
2,4560,cinnamon. Why didn't Cha get chocolate?,Cinnamon babka.,"I spilled some Chardonnay. So, what did you get?",What for?,I had to give it to the liquor store guy.,"Hey, what happened to your coat? And what is t...",Somebody put a cane on my foot. Just like the ...,"What, what happened to you?","S-s-somebody double parked, we couldn't help i...","How about a nice box of ""scram"".","Oh, no thanks."
3,8918,it's a pleasure to meet you.,"(Introducing the two) George, this is Nina.",Yeah!,It's been years!,Hi.,Nina?,Jerry?,Five' eight. Five' seven.,Really?,"Yeah, and a whole new me. I'm up two inches in...","Almost. (Notices his shoes) Hey, new Timberlands?"
4,5581,Hey.,(entering) Hey.,"Many times. You love velvet, you want to live ...",I've said that before?,"I know, you would drape yourself in velvet.","I gotta find a way to work this out, I love th...",We'll take a check please.,Ho ho. This bizarre ?harrod? experiment must end!,Not you.,I look like me and I'm working from the outsid...,He looks like you and he's working from the in...
...,...,...,...,...,...,...,...,...,...,...,...,...
965,856,It's the pesto of cities. So..?,Everybody's moving to Seattle.,Seattle.,"He's from a.. Yakima, right?","(Slightly embarrassed) Yes, a guy.",A guy?,"Elaine is having a ""houseguest."" She's picking...",Who?,I don't understand why he couldn't take a cab.,That's a tough minute. It's like waiting in th...,"(To Elaine) Oh, one more thing about the car. ..."
966,1071,That's pretty long.,About three years.,So how long did you live there?,"On my block, a lot of ah, people walk their do...","Well, thank you very much. I'm telling you, on...",It's just we want you to go.,"I know, it's not that.","Forget it. Go ahead, you'll have a good time.",What about this--,No.,Oh! I tell you what. How about if I come back ...
967,5846,(enthusiastic) Yeah. You get a couch. I get ri...,(not sure) I guess.,"No, the cushion's turned over.",But it's got a pee-stain on it.,"Yeah, sure. (big smile) Then my father will ha...",The one with the Poppie stain?!,(crafty) Not necessarily. Why don't you take b...,Now we have to buy a new couch?!,We have to replace the couch.,"Anyway, Jerry... Jerry?",And they think they're better than us?
968,3688,Why?,I don't know about defraying.,"Hey, guess what? The Drake broke up.",Hey.,"Don't worry, they'll make friends fast with th...",They don't know anybody in Chicago.,I hate the Drake! Maybe the whole thing was a ...,"Boy, I am really starting to dislike the Drake!",I know it is!,"Well, she can't keep it, it's not fair, that's...",He gave her all the gifts; he felt guilty.


In [ ]:
#save to csv 

back_translated_tenpercent_sample.to_csv('/content/drive/My Drive/Colab Notebooks/back_translated_tenpercent_sample.csv',index=False)